<a href="https://colab.research.google.com/github/robgon-art/trip_down_ml/blob/main/TripDown_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install face_recognition

In [ ]:
import face_recognition
import glob
import pandas as pd
import numpy as np
from PIL import Image

# Step 1: Read images, get face positions and embeddings
def get_face_data(image_path):
    image = face_recognition.load_image_file(image_path)

    # Convert the numpy array image to a PIL Image object
    img = Image.fromarray(image)
    # Get the current size
    width, height = img.size
    # Determine the scaling factor
    max_dimension = max(width, height)
    scaling_factor = 1024 / max_dimension
    # Calculate the new size
    new_width = int(width * scaling_factor)
    new_height = int(height * scaling_factor)
    # Resize the image
    resized_img = img.resize((new_width, new_height), Image.LANCZOS)
    # Convert the PIL Image back to a numpy array if needed
    resized_image_array = np.array(resized_img)

    face_locations = face_recognition.face_locations(resized_image_array, model="cnn")

    # Scale the face locations back up to the original image size
    scaled_face_locations = []
    for location in face_locations:
        top, right, bottom, left = location
        scaled_location = (
            int(top / scaling_factor),
            int(right / scaling_factor),
            int(bottom / scaling_factor),
            int(left / scaling_factor)
        )
        scaled_face_locations.append(scaled_location)

    # Generate encodings from the positions in the full-sized image
    encodings = face_recognition.face_encodings(image, known_face_locations=scaled_face_locations)
    return scaled_face_locations, encodings

file_path = "/content/drive/MyDrive/photos/Jen and Rob backyard.jpg"
positions, encs = get_face_data(file_path)
print(positions)

In [ ]:
from IPython.display import Image as display_Image

# Display the image
display_Image(filename=file_path, width=400)

In [ ]:
from PIL import Image
from IPython.display import display as display_function

# Assuming `file_path` is a variable containing the path to your image
# # And `positions` is a list of tuples with the face locations in the format (top, right, bottom, left)

# Load the image
image = Image.open(image_1_path)

# Loop through the face positions and display each cropped face
for position in face_locations_1:
    print(position)
    # Crop the image to the face location
    top, right, bottom, left = position
    face_image = image.crop((left, top, right, bottom))

    # Resize the face image
    face_image = face_image.resize((256, 256), Image.LANCZOS )

    # Display the face image
    display_function(face_image)

In [ ]:
import glob
file_paths = glob.glob("/content/drive/MyDrive/photos/*")
file_paths.sort()
print(len(file_paths))

In [ ]:
# Step 2: Store in DataFrame
df = pd.DataFrame(columns=['file_path', 'position', 'encoding_index'])
embeddings = []

for i, file_path in enumerate(file_paths):
    if i %10 == 0:
      print(i)
    positions, encs = get_face_data(file_path)
    for pos, enc in zip(positions, encs):
        new_row = pd.DataFrame({'file_path': [file_path],
                                'position': [pos],
                                'encoding_index': [len(embeddings)],
                                'label': [None]  # Initialize the label as None or a placeholder
                                })

        df = pd.concat([df, new_row], ignore_index=True)
        embeddings.append(enc)

# Convert embeddings to a NumPy array
embeddings = np.array(embeddings)

In [ ]:
df

In [ ]:
import matplotlib.pyplot as plt
import random

# Step 1: Select 20 random samples
sample_df = df.sample(20)

# Step 2: Create a 4x5 grid for thumbnails
fig, axes = plt.subplots(4, 5, figsize=(6, 5))

# Flatten the axes array for easy indexing
axes = axes.ravel()

# Step 3: Load images, resize, crop, and display
for i, (idx, row) in enumerate(sample_df.iterrows()):
    # Load the image
    image = Image.open(row['file_path'])

    # Crop to the face location
    position = row['position']
    top, right, bottom, left = position
    face_image = image.crop((left, top, right, bottom))

    # Resize to thumbnail (preserving aspect ratio)
    face_image.thumbnail((64, 64))

    # Display the thumbnail
    axes[i].imshow(face_image)
    axes[i].axis('off')

# Turn off any unused axes
for j in range(i + 1, 20):
    axes[j].axis('off')

plt.tight_layout()
plt.show()


In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
from PIL import Image as PILImage
import io
import gc

# Global variables to hold widgets
face_widgets = []
selection_buttons = []

thumb_size = 150

def load_and_display_faces(file_paths, positions):
    global face_widgets, selection_buttons

    # face_widgets.clear()
    # selection_buttons.clear()

    # for widget in face_widgets + selection_buttons:
    #     widget.close()
    face_widgets.clear()
    selection_buttons.clear()

    for i, (file_path, (top, right, bottom, left)) in enumerate(zip(file_paths, positions)):
        image = PILImage.open(file_path).convert('RGB')
        face = image.crop((left, top, right, bottom))
        face = face.resize((thumb_size, thumb_size), PILImage.LANCZOS)

        with io.BytesIO() as output:
            face.save(output, format="JPEG")
            face_bytes = output.getvalue()

        img_widget = widgets.Image(value=face_bytes, format='jpeg', width=thumb_size, height=thumb_size)
        face_widgets.append(img_widget)

        button = widgets.ToggleButton(description=str(i), value=False, icon='check')
        selection_buttons.append(button)

    return face_widgets, selection_buttons

def get_unlabeled_sample():
    unlabeled_df = df[df['label'].isnull()]
    sample_size = min(len(unlabeled_df), 24)
    return unlabeled_df.sample(sample_size)

def update_display(new_sample_df):
    global grid_layout

    gc.collect()

    file_paths = new_sample_df['file_path'].tolist()
    positions = new_sample_df['position'].tolist()
    face_widgets, selection_buttons = load_and_display_faces(file_paths, positions)

    for i in range(len(face_widgets)):
        # Get the VBox at the specific grid position and update its children
        grid_layout[i // 8, i % 8].children = [face_widgets[i], selection_buttons[i]]

def on_submit_clicked(b):
    global sample_df

    gc.collect()
    selected_indices = [int(button.description) for button in selection_buttons if button.value]

    if selected_indices:
        selected_df_indices = sample_df.iloc[selected_indices].index

        for idx in selected_df_indices:
            # Set the label for the current face
            df.loc[idx, 'label'] = name_text.value

            # Retrieve the encoding of the selected face
            face_encoding = embeddings[df.loc[idx, 'encoding_index']]

            # Compare with known face encodings
            matches = face_recognition.compare_faces(embeddings,
                                                     face_encoding,
                                                     tolerance=0.4) # strict

            # Update the label for all matching faces
            for match_idx, is_match in enumerate(matches):
                if is_match:
                    df.loc[df['encoding_index'] == match_idx, 'label'] = name_text.value

        # print("Set the label for", match_count, "images to", name_text.value)

    for button in selection_buttons:
        button.value = False

    clear_output(wait=True)
    sample_df = get_unlabeled_sample()
    update_display(sample_df)

def on_enter_pressed(_):
    on_submit_clicked(None)

# Create grid layout and controls layout placeholders
grid_layout = widgets.GridspecLayout(3, 8)
controls_layout = widgets.HBox(layout=widgets.Layout(justify_content='center'))

# Initialize with first batch of images
sample_df = get_unlabeled_sample()
file_paths = sample_df['file_path'].tolist()
positions = sample_df['position'].tolist()
face_widgets, selection_buttons = load_and_display_faces(file_paths, positions)

for i in range(len(face_widgets)):
    box = widgets.VBox([face_widgets[i], selection_buttons[i]], layout=widgets.Layout(margin='0 0 10px 0'))
    grid_layout[i // 8, i % 8] = box

# Set up the Text and Button widgets
name_text = widgets.Text(description="Name:")
name_text.on_submit(on_enter_pressed)
submit_button = widgets.Button(description="Submit")
submit_button.on_click(on_submit_clicked)

controls_layout.children = [name_text, submit_button]

# Display the initial UI
display(grid_layout)
display(controls_layout)


In [ ]:
# Count the occurrences of each label and sort them in descending order
label_counts = df['label'].value_counts().sort_values(ascending=False)

status = ""
for label, count in label_counts.items():
  status += label + " " + str(count) + ", "
if len(status) > 2:
  status = status[:-2]
print(status)

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image

def show_images(selected_label):

  # Filter the DataFrame for the selected label
  selected_df = df[df['label'] == selected_label]

  # Extract file paths and positions
  file_paths = selected_df['file_path'].tolist()
  positions = selected_df['position'].tolist()

  # Number of images and setting up grid dimensions
  num_images = len(file_paths)
  num_cols = 4
  num_rows = (num_images + num_cols - 1) // num_cols  # Calculate the required number of rows

  # Create the figure with subplots
  fig, axes = plt.subplots(num_rows, num_cols, figsize=(10, num_rows * 2.5))
  axes = axes.flatten()

  for i, (file_path, (top, right, bottom, left)) in enumerate(zip(file_paths, positions)):
      # print(file_path)
      image = Image.open(file_path).convert('RGB')

      # Crop the image to the face region
      face_region = (left, top, right, bottom)
      face = image.crop(face_region)

      # Resize the face to 100x100
      face_resized = face.resize((100, 100), Image.LANCZOS)

      axes[i].imshow(face_resized)
      axes[i].axis('off')

  # Turn off axes for any empty subplots
  for j in range(i + 1, num_cols * num_rows):
      axes[j].axis('off')

  plt.tight_layout()
  plt.show()

In [ ]:
df

In [ ]:
labels_set_by_user = ["Rob", "Jen"]
avg_encodings = {}

for label in labels_set_by_user:
    label_encodings = embeddings[df['label'] == label]
    avg_encodings[label] = np.mean(label_encodings, axis=0)

def find_match(encoding, avg_encodings, threshold=0.6):
    for label, avg_encoding in avg_encodings.items():
        if face_recognition.face_distance([avg_encoding], encoding)[0] < threshold:
            return label
    return None

df['predicted_label'] = None  # Initialize the column

for index, row in df.iterrows():
    current_encoding = embeddings[row['encoding_index']]
    predicted_label = find_match(current_encoding, avg_encodings)

    if predicted_label is not None:
        df.at[index, 'predicted_label'] = predicted_label

In [ ]:
df

In [ ]:
def find_images_with_all_labels(labels):
    # Create a DataFrame to hold file_paths that contain all labels
    valid_file_paths = pd.DataFrame(columns=['file_path'])

    # Get unique file_paths
    unique_file_paths = df['file_path'].unique()

    # Iterate through each file_path
    for file_path in unique_file_paths:
        # Get all rows for the current file_path
        rows = df[df['file_path'] == file_path]
        # Check if all specified labels are present in these rows
        if all(label in rows['predicted_label'].values for label in labels):
            # Add the file_path to the valid_file_paths DataFrame
            valid_file_paths = valid_file_paths.append({'file_path': file_path}, ignore_index=True)

    return valid_file_paths

labels_to_check = ['Rob', 'Jen']
valid_paths_df = find_images_with_all_labels(labels_to_check)

# Now you can use valid_paths_df to process images that contain all specified labels
valid_paths_df

In [ ]:
valid_paths_df

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image

def show_valid_images(valid_paths_df):
    # Extract file paths
    file_paths = valid_paths_df['file_path'].tolist()

    # Number of images and setting up grid dimensions
    num_images = len(file_paths)
    if num_images == 0:
        print("No valid images to display.")
        return

    num_cols = 4
    num_rows = (num_images + num_cols - 1) // num_cols  # Calculate the required number of rows

    # Create the figure with subplots
    fig, axes = plt.subplots(num_rows, num_cols, figsize=(12, num_rows * 2.5))
    axes = axes.flatten()

    for i, file_path in enumerate(file_paths):
        # Load the image
        image = Image.open(file_path).convert('RGB')

        # Display the image
        axes[i].imshow(image)
        axes[i].axis('off')

    # Turn off axes for any empty subplots
    for j in range(i + 1, num_cols * num_rows):
        axes[j].axis('off')

    plt.tight_layout()
    plt.show()

# Call the function with the dataframe of valid paths
show_valid_images(valid_paths_df)


In [ ]:
# prompt: convert valid_paths_df to a list

valid_paths_list = valid_paths_df['file_path'].tolist()
print(valid_paths_list)
